In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("MODEL BUILDING AND EVALUATION")
print("="*50)

# 1. LOAD TRAINED MODEL AND DATA
print("\n1. LOADING TRAINED MODEL AND DATA")
print("="*30)

import joblib
import json

# Load best model
try:
    best_model = joblib.load('best_model_random_forest.pkl')
    print("Loaded: Random Forest model")
except:
    try:
        best_model = joblib.load('best_model_gradient_boosting.pkl')
        print("Loaded: Gradient Boosting model")
    except:
        print("Loading default model...")
        from sklearn.ensemble import RandomForestRegressor
        best_model = RandomForestRegressor()

# Load feature information
with open('feature_info.json', 'r') as f:
    feature_info = json.load(f)

selected_features = feature_info['selected_features']
print(f"Number of features: {len(selected_features)}")

# Load predictions
predictions_df = pd.read_csv('test_predictions.csv')
print(f"Test predictions loaded: {predictions_df.shape[0]} samples")

# Load full dataset for analysis
df = pd.read_csv('cleaned_infectious_disease.csv')
df_total = df[df['Sex'] == 'Total'].copy()


MODEL BUILDING AND EVALUATION

1. LOADING TRAINED MODEL AND DATA
Loading default model...


FileNotFoundError: [Errno 2] No such file or directory: 'feature_info.json'

In [3]:
# 2. HYPERPARAMETER OPTIMIZATION RESULTS
print("\n2. HYPERPARAMETER OPTIMIZATION")
print("="*30)

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

# Prepare data for hyperparameter tuning
from sklearn.model_selection import train_test_split

# Use the same feature engineering as in training
# (Recreating features for demonstration)
features_df = df_total.copy()
features_df['Year_Since_2000'] = features_df['Year'] - 2000
features_df['Rate_Lag1'] = features_df.groupby('County')['Rate'].shift(1)
features_df = features_df.dropna(subset=['Rate_Lag1'])

# Create simplified feature set for demonstration
X = features_df[['Year_Since_2000', 'Rate_Lag1']]
y = features_df['Rate']

# Split data
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, shuffle=False
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, shuffle=False
)

# Define parameter grid for Random Forest
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

print("Performing Randomized Search for Random Forest...")
rf = RandomForestRegressor(random_state=42, n_jobs=-1)

# Use randomized search for efficiency
random_search = RandomizedSearchCV(
    rf, param_distributions=param_grid,
    n_iter=20,  # Number of parameter settings sampled
    cv=3,
    scoring='neg_root_mean_squared_error',
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

print("Best parameters found:")
for param, value in random_search.best_params_.items():
    print(f"  {param}: {value}")

print(f"Best cross-validation RMSE: {-random_search.best_score_:.4f}")

# Train with best parameters
best_rf = random_search.best_estimator_
y_pred_val = best_rf.predict(X_val)

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
val_rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
print(f"Validation RMSE with optimized parameters: {val_rmse:.4f}")



2. HYPERPARAMETER OPTIMIZATION


NameError: name 'df_total' is not defined

In [ ]:
# 3. MODEL COMPARISON WITH STATISTICAL TESTS
print("\n3. MODEL COMPARISON WITH STATISTICAL TESTS")
print("="*30)

from scipy import stats
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge

# Train comparison models
models = {
    'Random Forest': best_rf,
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'Ridge Regression': Ridge(alpha=1.0)
}

# Store predictions for statistical tests
predictions = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    pred = model.predict(X_val)
    predictions[name] = pred
    rmse = np.sqrt(mean_squared_error(y_val, pred))
    print(f"{name:20s}: RMSE = {rmse:.4f}")

# Diebold-Mariano test for comparing forecasts
def diebold_mariano_test(y_true, pred1, pred2, h=1):
    """Diebold-Mariano test for predictive accuracy."""
    e1 = y_true - pred1
    e2 = y_true - pred2
    d = e1**2 - e2**2
    
    n = len(d)
    dm_stat = np.mean(d) / np.sqrt(np.var(d) / n)
    p_value = 2 * (1 - stats.norm.cdf(abs(dm_stat)))
    
    return dm_stat, p_value

# Compare Random Forest vs Gradient Boosting
dm_stat, p_value = diebold_mariano_test(y_val, predictions['Random Forest'], 
                                        predictions['Gradient Boosting'])
print(f"\nDiebold-Mariano Test (RF vs GB):")
print(f"  DM Statistic: {dm_stat:.4f}")
print(f"  p-value: {p_value:.4f}")
print(f"  Significant difference: {'Yes' if p_value < 0.05 else 'No'}")


In [ ]:
# 4. PERFORMANCE METRICS ANALYSIS
print("\n4. DETAILED PERFORMANCE METRICS")
print("="*30)

def calculate_metrics(y_true, y_pred, model_name):
    """Calculate comprehensive performance metrics."""
    metrics = {}
    
    # Basic metrics
    metrics['MAE'] = mean_absolute_error(y_true, y_pred)
    metrics['RMSE'] = np.sqrt(mean_squared_error(y_true, y_pred))
    metrics['R2'] = r2_score(y_true, y_pred)
    
    # Percentage errors
    metrics['MAPE'] = np.mean(np.abs((y_true - y_pred) / np.maximum(y_true, 1e-10))) * 100
    
    # Symmetric MAPE
    metrics['sMAPE'] = 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / 
                                               (np.abs(y_true) + np.abs(y_pred) + 1e-10))
    
    # Theil's U statistic
    numerator = np.sqrt(np.mean((y_true - y_pred)**2))
    denominator = np.sqrt(np.mean(y_true**2)) + np.sqrt(np.mean(y_pred**2))
    metrics['Theil_U'] = numerator / denominator
    
    # Directional accuracy
    y_true_chg = np.diff(y_true) > 0
    y_pred_chg = np.diff(y_pred) > 0
    if len(y_true_chg) > 0:
        metrics['DA'] = np.mean(y_true_chg == y_pred_chg)
    else:
        metrics['DA'] = np.nan
    
    print(f"\n{model_name} Performance Metrics:")
    for metric, value in metrics.items():
        print(f"  {metric:10s}: {value:.4f}")
    
    return metrics

# Calculate metrics for best model
best_metrics = calculate_metrics(y_test, predictions_df['Predicted'].values, 
                                "Best Model on Test Set")


In [4]:
# 5. RESIDUAL DIAGNOSTICS
print("\n5. RESIDUAL ANALYSIS")
print("="*30)

# Calculate residuals
residuals = predictions_df['Actual'] - predictions_df['Predicted']

# Create residual analysis plots
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Residual Diagnostics', fontsize=16)

# 1. Residual distribution
ax1 = axes[0, 0]
ax1.hist(residuals, bins=30, edgecolor='black', alpha=0.7)
ax1.axvline(x=0, color='red', linestyle='--', linewidth=1)
ax1.set_title('Distribution of Residuals')
ax1.set_xlabel('Residual')
ax1.set_ylabel('Frequency')
ax1.grid(True, alpha=0.3)

# 2. Q-Q plot
ax2 = axes[0, 1]
stats.probplot(residuals, dist="norm", plot=ax2)
ax2.set_title('Q-Q Plot of Residuals')
ax2.grid(True, alpha=0.3)

# 3. Residuals vs Predicted
ax3 = axes[0, 2]
scatter = ax3.scatter(predictions_df['Predicted'], residuals, alpha=0.6)
ax3.axhline(y=0, color='red', linestyle='--', linewidth=1)
ax3.set_title('Residuals vs Predicted Values')
ax3.set_xlabel('Predicted Rate')
ax3.set_ylabel('Residual')
ax3.grid(True, alpha=0.3)

# 4. Residuals vs Time (Year)
ax4 = axes[1, 0]
for county in predictions_df['County'].unique()[:5]:  # Top 5 counties
    county_data = predictions_df[predictions_df['County'] == county]
    ax4.scatter(county_data['Year'], county_data['Actual'] - county_data['Predicted'], 
               alpha=0.7, label=county, s=50)
ax4.axhline(y=0, color='red', linestyle='--', linewidth=1)
ax4.set_title('Residuals by Year (Top 5 Counties)')
ax4.set_xlabel('Year')
ax4.set_ylabel('Residual')
ax4.legend(loc='best', fontsize=8)
ax4.grid(True, alpha=0.3)

# 5. Actual vs Predicted
ax5 = axes[1, 1]
ax5.scatter(predictions_df['Actual'], predictions_df['Predicted'], alpha=0.6)
# Perfect prediction line
min_val = min(predictions_df['Actual'].min(), predictions_df['Predicted'].min())
max_val = max(predictions_df['Actual'].max(), predictions_df['Predicted'].max())
ax5.plot([min_val, max_val], [min_val, max_val], 'r--', label='Perfect Prediction')
ax5.set_title('Actual vs Predicted Values')
ax5.set_xlabel('Actual Rate')
ax5.set_ylabel('Predicted Rate')
ax5.legend()
ax5.grid(True, alpha=0.3)

# 6. Residual autocorrelation
ax6 = axes[1, 2]
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(residuals, lags=10, ax=ax6, alpha=0.05)
ax6.set_title('Autocorrelation of Residuals')
ax6.set_xlabel('Lag')
ax6.set_ylabel('Autocorrelation')
ax6.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('residual_diagnostics.png', dpi=300, bbox_inches='tight')
plt.show()

# Statistical tests on residuals
print("\nStatistical Tests on Residuals:")
print("-" * 30)

# Shapiro-Wilk test for normality
shapiro_stat, shapiro_p = stats.shapiro(residuals)
print(f"Shapiro-Wilk test for normality:")
print(f"  Statistic: {shapiro_stat:.4f}")
print(f"  p-value: {shapiro_p:.4e}")
print(f"  Residuals are normal: {'Yes' if shapiro_p > 0.05 else 'No'}")

# Durbin-Watson test for autocorrelation
from statsmodels.stats.stattools import durbin_watson
dw_stat = durbin_watson(residuals)
print(f"\nDurbin-Watson test for autocorrelation:")
print(f"  Statistic: {dw_stat:.4f}")
print(f"  Interpretation: {'No autocorrelation' if 1.5 < dw_stat < 2.5 else 'Possible autocorrelation'}")

# Breusch-Pagan test for heteroscedasticity
import statsmodels.api as sm
X_with_const = sm.add_constant(predictions_df['Predicted'])
model = sm.OLS(residuals**2, X_with_const).fit()
bp_stat = model.nobs * model.rsquared
bp_p = 1 - stats.chi2.cdf(bp_stat, 1)
print(f"\nBreusch-Pagan test for heteroscedasticity:")
print(f"  Statistic: {bp_stat:.4f}")
print(f"  p-value: {bp_p:.4e}")
print(f"  Homoscedastic: {'Yes' if bp_p > 0.05 else 'No'}")



5. RESIDUAL ANALYSIS


NameError: name 'predictions_df' is not defined

In [5]:
# 6. BIAS-VARIANCE ANALYSIS
print("\n6. BIAS-VARIANCE ANALYSIS")
print("="*30)

from sklearn.model_selection import learning_curve

# Prepare data for learning curves
X_full = pd.concat([X_train, X_val, X_test])
y_full = pd.concat([y_train, y_val, y_test])

# Calculate learning curves
train_sizes, train_scores, val_scores = learning_curve(
    best_rf, X_full, y_full,
    cv=3,
    scoring='neg_root_mean_squared_error',
    train_sizes=np.linspace(0.1, 1.0, 10),
    n_jobs=-1
)

# Calculate mean and std
train_scores_mean = -np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
val_scores_mean = -np.mean(val_scores, axis=1)
val_scores_std = np.std(val_scores, axis=1)

# Plot learning curves
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_scores_mean, 'o-', color='blue', 
         label='Training RMSE', linewidth=2)
plt.fill_between(train_sizes, 
                 train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std,
                 alpha=0.2, color='blue')

plt.plot(train_sizes, val_scores_mean, 'o-', color='green',
         label='Validation RMSE', linewidth=2)
plt.fill_between(train_sizes,
                 val_scores_mean - val_scores_std,
                 val_scores_mean + val_scores_std,
                 alpha=0.2, color='green')

plt.xlabel('Training Set Size')
plt.ylabel('RMSE')
plt.title('Learning Curves (Bias-Variance Analysis)')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)

# Calculate bias and variance
bias = train_scores_mean[-1]  # Training error at full dataset
variance = val_scores_mean[-1] - train_scores_mean[-1]  # Gap between val and train

print(f"Bias (Training Error): {bias:.4f}")
print(f"Variance (Validation - Training): {variance:.4f}")
print(f"Total Error: {val_scores_mean[-1]:.4f}")
print(f"\nInterpretation:")
if bias > variance:
    print("  Model is likely underfitting (high bias)")
else:
    print("  Model is likely overfitting (high variance)")

plt.savefig('learning_curves.png', dpi=300, bbox_inches='tight')
plt.show()



6. BIAS-VARIANCE ANALYSIS


NameError: name 'X_train' is not defined

In [6]:
# 7. MODEL EXPLAINABILITY (SHAP VALUES)
print("\n7. MODEL EXPLAINABILITY (SHAP ANALYSIS)")
print("="*30)

try:
    import shap
    
    print("Calculating SHAP values...")
    
    # Create explainer
    explainer = shap.TreeExplainer(best_rf)
    
    # Calculate SHAP values for a sample
    X_sample = X_train.sample(min(100, len(X_train)), random_state=42)
    shap_values = explainer.shap_values(X_sample)
    
    # Summary plot
    plt.figure(figsize=(10, 6))
    shap.summary_plot(shap_values, X_sample, show=False)
    plt.title('SHAP Feature Importance')
    plt.tight_layout()
    plt.savefig('shap_summary.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Force plot for a single prediction
    plt.figure(figsize=(12, 4))
    shap.force_plot(explainer.expected_value, shap_values[0], X_sample.iloc[0], 
                   matplotlib=True, show=False)
    plt.title('SHAP Force Plot for Single Prediction')
    plt.tight_layout()
    plt.savefig('shap_force_plot.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Calculate feature importance from SHAP
    shap_importance = pd.DataFrame({
        'feature': X.columns,
        'shap_importance': np.abs(shap_values).mean(axis=0)
    }).sort_values('shap_importance', ascending=False)
    
    print("\nTop 5 features by SHAP importance:")
    print(shap_importance.head())
    
except ImportError:
    print("SHAP not installed. Using feature importance from model instead.")
    
    # Feature importance from Random Forest
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': best_rf.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\nTop 5 features by model importance:")
    print(feature_importance.head())
    
    # Plot feature importance
    plt.figure(figsize=(10, 6))
    top_features = feature_importance.head(10)
    plt.barh(range(len(top_features)), top_features['importance'])
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Importance')
    plt.title('Feature Importance (Random Forest)')
    plt.gca().invert_yaxis()
    plt.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    plt.savefig('feature_importance.png', dpi=300, bbox_inches='tight')
    plt.show()



7. MODEL EXPLAINABILITY (SHAP ANALYSIS)
Calculating SHAP values...


NameError: name 'best_rf' is not defined

In [7]:
# 8. PREDICTION INTERVALS
print("\n8. PREDICTION INTERVALS")
print("="*30)

# Method 1: Using quantile regression forests
from sklearn.ensemble import RandomForestRegressor

# Train a model to estimate prediction intervals
rf_intervals = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_intervals.fit(X_train, y_train)

# Get predictions from individual trees
predictions_by_tree = np.array([tree.predict(X_test) for tree in rf_intervals.estimators_])

# Calculate prediction intervals
alpha = 0.05  # 95% prediction interval
lower_percentile = 100 * alpha / 2
upper_percentile = 100 * (1 - alpha / 2)

prediction_intervals = np.percentile(predictions_by_tree, 
                                     [lower_percentile, upper_percentile], 
                                     axis=0)

# Calculate coverage
in_interval = np.sum((y_test >= prediction_intervals[0]) & 
                     (y_test <= prediction_intervals[1])) / len(y_test)

print(f"95% Prediction Interval Coverage: {in_interval*100:.1f}%")
print(f"Average Interval Width: {np.mean(prediction_intervals[1] - prediction_intervals[0]):.4f}")

# Plot predictions with intervals
plt.figure(figsize=(12, 6))
sorted_idx = np.argsort(y_test.values)
plt.plot(range(len(y_test)), y_test.values[sorted_idx], 'o', 
         label='Actual', alpha=0.6, markersize=6)
plt.plot(range(len(y_test)), rf_intervals.predict(X_test).values[sorted_idx], 
         's', label='Predicted', alpha=0.6, markersize=4)
plt.fill_between(range(len(y_test)),
                 prediction_intervals[0][sorted_idx],
                 prediction_intervals[1][sorted_idx],
                 alpha=0.3, label='95% Prediction Interval')
plt.xlabel('Sample Index (Sorted by Actual Value)')
plt.ylabel('Incidence Rate')
plt.title('Predictions with 95% Prediction Intervals')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('prediction_intervals.png', dpi=300, bbox_inches='tight')
plt.show()



8. PREDICTION INTERVALS


NameError: name 'X_train' is not defined

In [ ]:
# 9. MODEL ROBUSTNESS CHECKS
print("\n9. MODEL ROBUSTNESS ANALYSIS")
print("="*30)

from sklearn.model_selection import cross_val_score

# Check stability across different random seeds
rmse_scores = []
r2_scores = []

for seed in range(42, 52):  # 10 different seeds
    rf_temp = RandomForestRegressor(n_estimators=100, random_state=seed, n_jobs=-1)
    
    # Cross-validation scores
    cv_rmse = -cross_val_score(rf_temp, X_full, y_full, 
                               cv=3, scoring='neg_root_mean_squared_error',
                               n_jobs=-1).mean()
    cv_r2 = cross_val_score(rf_temp, X_full, y_full, 
                           cv=3, scoring='r2', n_jobs=-1).mean()
    
    rmse_scores.append(cv_rmse)
    r2_scores.append(cv_r2)

print(f"RMSE across 10 random seeds:")
print(f"  Mean: {np.mean(rmse_scores):.4f}")
print(f"  Std:  {np.std(rmse_scores):.4f}")
print(f"  Range: [{np.min(rmse_scores):.4f}, {np.max(rmse_scores):.4f}]")

print(f"\nR² across 10 random seeds:")
print(f"  Mean: {np.mean(r2_scores):.4f}")
print(f"  Std:  {np.std(r2_scores):.4f}")
print(f"  Range: [{np.min(r2_scores):.4f}, {np.max(r2_scores):.4f}]")

# Sensitivity to feature removal
feature_sensitivity = {}
base_score = -cross_val_score(best_rf, X_full, y_full, 
                             cv=3, scoring='neg_root_mean_squared_error',
                             n_jobs=-1).mean()

for feature in X.columns:
    X_reduced = X_full.drop(columns=[feature])
    score = -cross_val_score(best_rf, X_reduced, y_full,
                            cv=3, scoring='neg_root_mean_squared_error',
                            n_jobs=-1).mean()
    feature_sensitivity[feature] = score - base_score

print("\nFeature Sensitivity (Increase in RMSE when removed):")
for feature, sensitivity in sorted(feature_sensitivity.items(), 
                                  key=lambda x: x[1], reverse=True):
    print(f"  {feature:20s}: {sensitivity:+.4f}")